In [3]:
using Random
using Distributions

include("simulate.jl")
include("data.jl")

generateData (generic function with 3 methods)

In [131]:
using FLoops

In [133]:
@floop for (x, y) in zip(1:3, 1:2:6)
   a = x + y
   b = x - y
   @reduce c += a
   @reduce d += b
end
(c, d)

(15, -3)

In [135]:
"""
DATA CONFIG
"""
alpha = 0.05

# data
B  = 100    # num. coverage probabilities per boxplot
S  = 1800   # num. samples per coverage probability
nx = 8      # size of group 1
ny = 8     # size of group 2
dtype = Float32
seed = 123

"""
POPULATION SETTINGS
"""

Random.seed!(seed)
distrTypeX = Normal{dtype}
muX = dtype.(round.(rand(Uniform(0, 1), B), digits=4))
sdX = 2 * muX
paramsX = (muX, sdX)

distrTypeY = Normal{dtype}
paramsY = (2 .+ muX, sdX)

x, y, deltas, distrX, distrY = generateData(B, S, nx, ny,
                                            distrTypeX, paramsX, distrTypeY, paramsY,
                                            dtype, seed)

LoadError: UndefVarError: generateData not defined

In [ ]:
include("simulate.jl")

mc_size = try
    (binomial(nx+ny, nx) > 45_000) ? 10_000 : 0
catch
    0
end

@time begin
simulate_perm(x, y, deltas, distrX, distrY,
              alpha=alpha, mc_size=mc_size, save_csv=true)
end

results = Any[(0.9, 3.1916518750843292), (0.8933333333333333, 1.552045956092695), (0.8944444444444445, 2.58581600350156), (0.8977777777777778, 1.7987317674636094), (0.9033333333333333, 0.8838585799830667), (0.9066666666666666, 1.165940473613707), (0.9061111111111111, 1.5029058650019096), (0.9133333333333333, 3.0398432360306247), (0.8994444444444445, 0.349165674251206), (0.9027777777777778, 0.4395399017045684), (0.8977777777777778, 2.4097164606142574), (0.91, 0.4759951734777365), (0.92, 0.1120305417623851), (0.8988888888888888, 1.2284029987036769), (0.9016666666666666, 3.261435922583499), (0.9038888888888889, 3.3439004231728853), (0.8894444444444445, 2.730601525430088), (0.9111111111111111, 0.6481651337733737), (0.9055555555555556, 1.0344733922405476), (0.9055555555555556, 0.5259823341244821), (0.9027777777777778, 3.136080240210462), (0.8994444444444445, 1.2421682622503578), (0.9222222222222223, 0.46326366463456403), (0.895, 3.2669331845260583), (0.9144444444444444, 0.19838578645996227)

In [28]:
simulate_bootstrap(x, y, deltas, distrX, distrY,
                   alpha=alpha, nsamples=10_000, save_csv=false)

results = Any[(1.0, 4.3203465133870775)]
results = Any[(1.0, 4.384213209152222)]


In [20]:
include("simulate.jl")

simulate_t(x, y, deltas, distrX, distrY,
           alpha=alpha, save_csv=false)

results = Any[(0.9444444444444444, 2.2061780541731277)]
results = Any[(0.9466666666666667, 2.2326594997223967)]


In [1]:
include("intervals/permutation.jl")

testStatDistr (generic function with 1 method)

In [5]:
using Distributions, Random

In [10]:
Dx = Normal(0, 1)
Dy = Normal(0, 1)
Random.seed!(123)
nx = ny = 5
x = rand(Dx, nx)
y = rand(Dy, ny)

5-element Vector{Float64}:
  0.9809798121241488
  0.0799568295050599
  1.5491245530427917
 -1.3416092408832219
  0.41216163468296796

In [11]:
include("util.jl")
px, py = partition(nx, ny)

([1 1 … 5 6; 2 2 … 7 7; … ; 4 4 … 9 9; 5 6 … 10 10], [10 10 … 6 5; 9 9 … 4 4; … ; 7 7 … 2 2; 6 5 … 1 1])

In [31]:
include("intervals/bootstrap.jl")
include("intervals/t.jl")

t (generic function with 1 method)

In [97]:
n = 0
for column in 1:10_000
    n += f(px[:,column], collect(1:5))
end
@show n / 10_000
@show 1/252

n / 10000 = 0.0037
1 / 252 = 0.003968253968253968


0.003968253968253968

In [125]:
Dx = Gamma(10, 0.9)
Dy = Gamma(10, 1)
@show mean(Dx), mean(Dy)
@show var(Dx), var(Dy)

(mean(Dx), mean(Dy)) = (9.0, 10.0)
(var(Dx), var(Dy)) = (8.100000000000001, 10.0)


(8.100000000000001, 10.0)

In [139]:
idxs = [(i, j) for i in 1:B for j in 1:S];

In [ ]:
covered ./ S

In [180]:
# do exact example on small nx = ny = 3

Dx = Normal(0, 1)
Dy = Normal(2, 1)
delta = mean(Dx) - mean(Dy)

B = 1
S = 1800
nx, ny = 10, 10
px, py = partition(nx, ny)
mc = 10_000


idxs = [(i, j) for i in 1:B for j in 1:S]
covered = zeros(6, B)
widths = zeros(6, B)
Random.seed!(123)

@time @floop ThreadedEx() for (i, j) in idxs
    # x = rand(Dx[i], nx)
    # y = rand(Dy[i], ny)
    x = rand(Dx, nx)
    y = rand(Dy, ny)

    lo, hi = permInterval(x, y, nothing, (px, py, mc), true, 0.05, twoSided, twoSided)
    covered[1, i] += (lo <= delta <= hi)
    #width[1, i] += hi - lo

    """
    lo, hi = permInterval(x, y, nothing, (px, py, mc), false, 0.05, twoSided, twoSided)
    coverages[2] += (lo <= delta <= hi)

    lo, hi = permInterval(x, y, nothing, (px, py, mc), true, 0.05, greater, smaller)
    coverages[3] += (lo <= delta <= hi)

    lo, hi = permInterval(x, y, nothing, (px, py, mc), false, 0.05, greater, smaller)
    coverages[4] += (lo <= delta <= hi)

    lo, hi = bootstrap(x, y, 0.05, false, 10_000)
    coverages[5] += (lo <= delta <= hi)

    lo, hi = tconf(x, y, 0.05, false)
    coverages[6] += (lo <= delta <= hi)
    """
end
covered ./ S

 25.307327 seconds (550.73 M allocations: 181.530 GiB, 24.80% gc time, 0.26% compilation time)


6×1 Matrix{Float64}:
 0.8938888888888888
 0.0
 0.0
 0.0
 0.0
 0.0

In [127]:
# do exact example on small nx = ny = 3
@time begin
    Dx = Gamma(10, 0.9)
    Dy = Gamma(10, 1)
    delta = mean(Dx) - mean(Dy)
    nx, ny = 10, 10
    px, py = partition(nx, ny)
    S = 50
    mc = 10_000
    coverages = [0, 0, 0, 0, 0, 0]
    Random.seed!(123)
    for _ in 1:S
        x = rand(Dx, nx)
        y = rand(Dy, ny)
        
        lo, hi = permInterval(x, y, nothing, (px, py, mc), true, 0.05, twoSided, twoSided)
        coverages[1] += (lo <= delta <= hi)
        
        lo, hi = permInterval(x, y, nothing, (px, py, mc), false, 0.05, twoSided, twoSided)
        coverages[2] += (lo <= delta <= hi)
        
        lo, hi = permInterval(x, y, nothing, (px, py, mc), true, 0.05, greater, smaller)
        coverages[3] += (lo <= delta <= hi)
        
        lo, hi = permInterval(x, y, nothing, (px, py, mc), false, 0.05, greater, smaller)
        coverages[4] += (lo <= delta <= hi)
        
        lo, hi = bootstrap(x, y, 0.05, false, 10_000)
        coverages[5] += (lo <= delta <= hi)
        
        lo, hi = tconf(x, y, 0.05, false)
        coverages[6] += (lo <= delta <= hi)
    end
end
coverages ./ S

 10.672367 seconds (82.13 M allocations: 21.251 GiB, 4.84% gc time, 0.01% compilation time)


6-element Vector{Float64}:
 0.9
 0.9
 0.92
 0.92
 0.92
 1.0

In [ ]:
Alex Han
InvertingPermutationTests2022